<a href="https://colab.research.google.com/github/nealshiyekar/Deepfake-Detectoin/blob/main/GradioApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
!pip install torch
!pip install facenet-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.4/314.4 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 801.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.7 MB/s et

In [ ]:
import cv2
import csv
import pandas as pd
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.transforms import v2
import torchvision.io as io
from torchvision.io import read_image
import math
import os
import pickle
import PIL
from PIL import Image
import numpy as np
import random
import re
import shutil
IMAGE_SIZE = 112

In [ ]:
def CaptureFrames(vid_path, imageset):
  if os.path.exists("/content/"+ imageset +"/extracted") == True:
    shutil.rmtree("/content/" + imageset + "/extracted")

  output_folder = os.mkdir("/content/" + imageset + "/extracted")


  video_path = vid_path
  vidcap = cv2.VideoCapture(video_path)
  success, image = vidcap.read()

  if success == False:
    print("cound not read video frame")


    #Gets number of frames in the video
  num_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

  print(video_path + ' has ' + str(num_frames) + ' frames')

  next_frame = random.randrange(num_frames - 50)

  for i in range(50):
    #It's possible for the same random number to be generated, but because each video has hundreds of
    #frames, this is too uncommon to have a noticeable impact

    next_frame_str = str(next_frame).zfill(5)
    #If next_frame is an integer that is less than 5 digits long, this fills digits to the left with 0s

    vidcap.set(cv2.CAP_PROP_POS_FRAMES, next_frame)
    success, image = vidcap.read()

    image_name = "/content/" + imageset + "/extracted/" + f"poss_deepfake_frame{next_frame_str}.jpg"
    #Every filename has the format "Youtube-real_idABCDE_frameFGHIJ.jpg"

    cv2.imwrite(image_name, image)
    next_frame = next_frame + 1
    print(image_name)

# 'poss_deepfake_vid_frames' , or 'realimage'

def cropPredFrames(imageset):
  # crop
  # If required, create a face detection pipeline using MTCNN:
  mtcnn = MTCNN(image_size=IMAGE_SIZE, margin=35)

  # Create an inception resnet (in eval mode):
  resnet = InceptionResnetV1(pretrained='vggface2').eval()

  if os.path.exists("/content/" + imageset + "/CroppedFrames") == True:
    shutil.rmtree("/content/" + imageset + "/CroppedFrames")

  os.mkdir("/content/" + imageset + "/CroppedFrames")

  list_frames = os.listdir("/content/" + imageset + "/extracted")
  list_frames = sorted(list_frames)

  for frame in list_frames:
    image = Image.open("/content/" + imageset + "/extracted/" + frame)

    print(frame)

    # Get cropped and prewhitened image tensor
    image_cropped = mtcnn(image, save_path="/content/" + imageset + "/CroppedFrames/" + frame)


In [ ]:
def getFinalEmbeddings(imageset):
  if os.path.exists("/content/" + imageset + "/Embeddings") == True:
    shutil.rmtree("/content/" + imageset + "/Embeddings")

  embeddings_dict = {}
  # embeddings_folder = os.mkdir("/content/" + imageset + "Outputs/" + imageset + "Embeddings")

  #model = GhostFaceNetsV2(image_size=IMAGE_SIZE, width=1, dropout=0.)
  #model = GhostFaceNetsV2(image_size=IMAGE_SIZE, num_classes=3, width=1, dropout=0.)
  #model = GhostFaceNetsV2(image_size=IMAGE_SIZE, num_classes=0, width=1, dropout=0.)
  #model = GhostFaceNetsV2(image_size=IMAGE_SIZE, width=1, dropout=0.)

  model = InceptionResnetV1(pretrained='vggface2')

  model.eval()

  #list_frames = os.listdir("/content/" + imageset + "Outputs/" + imageset + "Frames")
  list_frames = os.listdir("/content/" + imageset + "/CroppedFrames")
  list_frames = sorted(list_frames)


  for frame in list_frames:
    #image = Image.open("/content/" + imageset + "Outputs/" + imageset + "Frames/" + frame)
    image = Image.open("/content/" + imageset + "/CroppedFrames/" + frame)
    transform = v2.Compose([v2.ToImage(), v2.Resize((IMAGE_SIZE, IMAGE_SIZE))])

    img_tensor = transform(image)
    img_tensor = img_tensor.float()
    img_tensor = img_tensor.unsqueeze(0)

    with torch.no_grad():
      embedding = model(img_tensor)
      embeddings_dict[frame] = embedding



      #print(embedding)
      #print(embedding.shape)

  #for frame, embedding in embeddings_dict.items():
  #  print(f"Frame: {frame}, Embedding: {embedding}")

  return embeddings_dict, list_frames

  # with open("/content/" + imageset + "Outputs/" + imageset + "Embeddings/" + imageset + "Embeddings.csv", "w", newline="") as csvFile:
  #   writer = csv.writer(csvFile)
  #   #writer.writerow(["Frame", "Embedding"])
  #   writer.writerow(["CroppedFrame", "Embedding"])
  #   for frame, embedding in embeddings_dict.items():
  #     writer.writerow([frame, ",".join(map(str, embedding))])


In [ ]:
def deepfakeImgImgPred(poss_deepfake, img_real):
  if os.path.exists("/content/realimage/extracted") == True:
    shutil.rmtree("/content/realimage/extracted")
  output_folder = os.mkdir("/content/realimage/extracted")
  real_img = Image.open(img_real)
  real_img = real_img.save("/content/realimage/extracted/img_real.jpg")
  poss_deepfake = Image.open(poss_deepfake)
  poss_deepfake = poss_deepfake.save("/content/realimage/extracted/poss_deepfake.jpg")
  cropPredFrames("realimage")
  # embeddings of each image
  imgEmbedding, frames = getFinalEmbeddings("realimage")
  #euclidean distance
  #torch.cdist(x1, x2, p=2.0, compute_mode='use_mm_for_euclid_dist_if_necessary')
  edist = torch.cdist(imgEmbedding[frames[0]],imgEmbedding[frames[1]],2.0)
  edist.shape
  print(edist.item())
  return str(edist.item())
def deepfakeVidImgPred(poss_deepfake, img_real):
  # capture frames from video
  # crop video frames and image
  # get embeddings for video frames and image
  # average video frame embeddings
  # euclidean distance between vid embeddings and image
  # give pred based on threshold
  if os.path.exists("/content/poss_deepfake_vid_frames") == True:
    shutil.rmtree("/content/poss_deepfake_vid_frames")
  folder = os.mkdir("/content/poss_deepfake_vid_frames")

  if os.path.exists("/content/poss_deepfake_vid_frames/extracted") == True:
    shutil.rmtree("/content/poss_deepfake_vid_frames/extracted")
  output_folder = os.mkdir("/content/poss_deepfake_vid_frames/extracted")

  CaptureFrames(poss_deepfake, "poss_deepfake_vid_frames")
  if os.path.exists("/content/realimage/extracted") == True:
    shutil.rmtree("/content/realimage/extracted")
  output_folder = os.mkdir("/content/realimage/extracted")
  real_img = Image.open(img_real)
  real_img = real_img.save("/content/realimage/extracted/img_real.jpg")
  cropPredFrames("poss_deepfake_vid_frames")
  cropPredFrames("realimage")
  vidEmbeddings,_ = getFinalEmbeddings("poss_deepfake_vid_frames")
  imgEmbedding,frames = getFinalEmbeddings("realimage")
  sum = torch.zeros(1, 512)
  for embedding in vidEmbeddings:
    sum = sum + vidEmbeddings[embedding]
  avg_vid_embedding = sum / 50
  edist = torch.cdist(imgEmbedding[frames[0]],avg_vid_embedding,2.0)
  print(edist)
  return str(edist.item())

def deepfakeVidVidPred(poss_deepfake, vid_real):
  if os.path.exists("/content/poss_deepfake_vid_frames") == True:
    shutil.rmtree("/content/poss_deepfake_vid_frames")
  folder = os.mkdir("/content/poss_deepfake_vid_frames")

  if os.path.exists("/content/realimage") == True:
    shutil.rmtree("/content/realimage")
  folder = os.mkdir("/content/realimage")

  if os.path.exists("/content/poss_deepfake_vid_frames/extracted") == True:
    shutil.rmtree("/content/poss_deepfake_vid_frames/extracted")
  output_folder = os.mkdir("/content/poss_deepfake_vid_frames/extracted")

  if os.path.exists("/content/realimage/extracted") == True:
    shutil.rmtree("/content/realimage/extracted")
  output_folder = os.mkdir("/content/realimage/extracted")
  CaptureFrames(poss_deepfake, "poss_deepfake_vid_frames")
  CaptureFrames(vid_real, "realimage")
  cropPredFrames("poss_deepfake_vid_frames")
  cropPredFrames("realimage")
  possdfvidEmbeddings,_ = getFinalEmbeddings("poss_deepfake_vid_frames")
  realvidEmbeddings,_ = getFinalEmbeddings("realimage")
  sum = torch.zeros(1, 512)
  for embedding in possdfvidEmbeddings:
    sum = sum + possdfvidEmbeddings[embedding]
  avg_dfvid_embedding = sum / 50
  realsum = torch.zeros(1, 512)
  for embedding in realvidEmbeddings:
    realsum = realsum + realvidEmbeddings[embedding]
  avg_real_embedding = realsum / 50
  edist = torch.cdist(avg_dfvid_embedding,avg_real_embedding,2.0)
  print(edist)
  return str(edist.item())

def deepfakePredHelper(input_type, poss_deepfake, img_real, input_format):

  if os.path.exists("/content/realimage") == True:
    shutil.rmtree("/content/realimage")
  folder = os.mkdir("/content/realimage")


  if input_type == "Image/Image":
    pred = deepfakeImgImgPred(poss_deepfake, img_real)
  if input_format == "Video/Image":
    pred = deepfakeVidImgPred(poss_deepfake, img_real)
  if input_format == "Image/Video":
    pred = deepfakeVidImgPred(img_real, poss_deepfake)
  if input_format == "Video/Video":
    pred = deepfakeVidVidPred(poss_deepfake, img_real)
  return pred


In [ ]:
!wget https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_linux_amd64
!cp '/content/frpc_linux_amd64' '/usr/local/lib/python3.10/dist-packages/gradio'

--2024-05-02 02:17:57--  https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_linux_amd64
Resolving cdn-media.huggingface.co (cdn-media.huggingface.co)... 13.33.21.15, 13.33.21.96, 13.33.21.8, ...
Connecting to cdn-media.huggingface.co (cdn-media.huggingface.co)|13.33.21.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11374592 (11M) [binary/octet-stream]
Saving to: ‘frpc_linux_amd64.1’

frpc_linux_amd64.1  100%[===================>]  10.85M  --.-KB/s    in 0.08s   

2024-05-02 02:17:57 (135 MB/s) - ‘frpc_linux_amd64.1’ saved [11374592/11374592]



In [ ]:
import gradio as gr


# put the overall function here
def predict(input_type, real, possible_deepfake):
  if input_type == "Image/Image" or input_type == "Image/Video":
        return "real"
  elif input_type == "Video/Image" or input_type == "Video/Video":
        return "fake"
  return "real"

inputs = [
    gr.Dropdown(choices=["Image/Image", "Video/Image", "Image/Video", "Video/Video"], label="Input Type", info="Choose type of inputs"),
    "file",
    "file"
]
outputs = ["textbox"]

demo = gr.Interface(
    fn=deepfakePredHelper,
    inputs = inputs,
    outputs= outputs,
    title="Deepfake Detection",
    description="Upload images or videos to determine if it's real or fake.",
    theme="soft",
    #live=True,
    allow_flagging="never",


    css = """
          .gradio-interface {
          font-family: Arial, sans-serif;
          background-color: #f0f0f0; /* Updated background color */
          padding: 20px; /* Added padding for spacing */
      }

      .gradio-input, .gradio-output {
          border-radius: 10px;
          box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.1);
          margin-bottom: 20px;
          background-color: #ffffff;
          padding: 20px; /* Added padding for spacing */
      }

      .gradio-input-title, .gradio-output-title {
          font-size: 20px;
          font-weight: bold;
          color: #333333;
          margin-bottom: 10px; /* Added margin for spacing */.
      }

      .output_textbox textarea {
          width: 100%;
          height: 150px;
          padding: 10px;
          border-radius: 5px;
          border: 1px solid #ced4da;
          font-size: 16px;
          resize: vertical;
      }

      .output_textbox label {
          display: block;
          margin-bottom: 5px;
      }
      """

)



# Launch the interface
demo.launch(debug=True)


/usr/local/lib/python3.10/dist-packages/gradio/utils.py:953: UserWarning: Expected 4 arguments for function <function deepfakePredHelper at 0x7d9349c49bd0>, received 3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:957: UserWarning: Expected at least 4 arguments for function <function deepfakePredHelper at 0x7d9349c49bd0>, received 3.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://6c87bc082110f438d7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/gradio/helpers.py:946: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/PIL/JpegImagePlugin.py", line 650, in _save
    rawmode = RAWMODE[im.mode]
KeyError: 'RGBA'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 270, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1847, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1433, in call_function
    prediction = a